# Webscpraping

In [1]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

## CoronaTracker

In [2]:
LANGUAGE = 'de'
LIMIT = '999'
URL = 'https://api.coronatracker.com/news/trending?limit=' + \
    LIMIT+'&offset=0&countryCode=&country=&language='+LANGUAGE

In [3]:
# Make a GET request to the url
request = requests.get(url=URL)

# Response as json
data = request.json()

In [4]:
for head in data:
    if head == 'total':
        head_t = head
        value = data[head]
    elif head == 'items':
        content = pd.DataFrame(data['items'])

        content[head_t] = value

content.to_csv('/Users/andang/Documents/workspace/wirvsvirus/data/coronatracker.csv', sep = "\t", encoding = 'utf8')

## RKI

In [5]:
def get_content(URL):
    # Make the GET request to a url
    request = requests.get(URL)
    # Extract the content
    content = request.content
    # Create a soup object
    soup = BeautifulSoup(content)

    return soup


def get_linklist(soup):
    link_list = []

    class_text = soup.find('div', attrs={'id': 'main'})

    for a in class_text.find_all('a', href=True):
        link = a['href']
        if link[:2] == 'DE':
            link_list.append('https://www.rki.de/'+link)
        elif link[:2] == '/D':
            link_list.append('https://www.rki.de'+link)
        elif link[:2] == 'Sh':
            link_list.append('https://www.rki.de/'+link)
        elif link[:2] == 'co':
            link_list.append('https://www.rki.de/'+link)
        else:
            link_list.append('https://www.rki.de/'+link)
    
    return(link_list)

In [6]:
URL = 'https://www.rki.de/SharedDocs/FAQ/NCOV2019/FAQ_Liste.html'

In [7]:
content = get_content(URL)
class_text = content.find_all('div', attrs={'class': 'alt-accordion-box-box'})
actuall_date = content.find_all('p', attrs={'class':'date'})

In [8]:
date_list = []
question_list = []
answer_list = []

In [9]:
for date in actuall_date:
    date_dict = {}
    date_dict['Stand'] = date.text[7:]
    date_list.append(date_dict)

In [10]:

for question in class_text:
    faq_dict = {}
    q = question.find('h2').text
    faq_dict['frage'] = q
    question_list.append(faq_dict)
    
for faq_text in class_text:
    faq_dict = {}
    a = faq_text.find('p').text
    faq_dict['antwort'] = a
    answer_list.append(faq_dict)


In [11]:
df1 = pd.DataFrame(answer_list)
df2 = pd.DataFrame(question_list)
df3 = pd.DataFrame(date_list)

pd.concat([df1,df2,df3], axis=1).to_csv('/Users/andang/Documents/workspace/wirvsvirus/data/rki_faq.csv', sep = "\t", encoding = 'utf8')